This is heavily inspired by the first chapter from [Refactoring: Ruby Edition](https://www.goodreads.com/book/show/11560939-refactoring?from_search=true) by [Martin Fowler](https://twitter.com/martinfowler).

I read the chapter, followed the examples and then closed the book and implemented this (this is how I learn). I include some notes below, but for a discussion of the underlying concepts (and how to integrate applying them into your or your team's workflow), you should get the book.

In [2]:
load './micro_test.rb'

true

> If you encounter a situation where you need to add a feature to a program, but the code is structured in such a way as to make it inconvenient, you should first refactor the program to make it easy to add the feature and then add the feature.

> But you should not start making changes unless you have a good set of tests documenting current behavior. Refactoring poorly written code without tests is like walking onto a minefield and hoping for the best. 

Tests is where we will start then. Here is the initial version of the code that we will refactor.

In [3]:
class Weapon
  WOODEN = 0
  BRONZE = 1
  ADAMANTIUM = 2
  
  attr_reader :name, :type
  
  def initialize(name, type)
    @name, @type = name, type
  end
end

class Inventory
  attr_reader :weapons
  
  def initialize
    @weapons = []
  end
  
  def add(weapon)
    @weapons << weapon
  end
end

class Ork
  attr_reader :name, :inventory
  
  def initialize(name)
    @name = name
    @inventory = Inventory.new
  end
  
  def summary
    result = "This is a scruffy looking Ork, that goes by the name of #{name}\n"
    result += "It carries the following weapons:\n"
    
    total_damage, total_bonus_damage_against_armored = 0, 0
    inventory.weapons.each do |weapon|
      this_bonus = 0
      case weapon.type
      when Weapon::WOODEN
        this_damage = 2
      when Weapon::BRONZE
        this_damage = 4
      when Weapon::ADAMANTIUM
        this_damage = 3
        this_bonus = 4
      end
        
      result += "- #{weapon.name}, damage: #{this_damage}, bonus against armored: #{this_bonus}\n"
      total_damage += this_damage
      total_bonus_damage_against_armored += this_bonus
    end
    
    result += "It can deal a total damage of #{total_damage} "
    result += "with a #{total_bonus_damage_against_armored} bonus against armored.\n"
  end
end

:summary

Let's see what our code does

In [4]:
ork = Ork.new('Richard')
ork.inventory.add(Weapon.new('Blade of Fury', 0))
ork.inventory.add(Weapon.new('Knife of Summoning', 2))
puts ork.summary()

This is a scruffy looking Ork, that goes by the name of Richard
It carries the following weapons:
- Blade of Fury, damage: 2, bonus against armored: 0
- Knife of Summoning, damage: 3, bonus against armored: 4
It can deal a total damage of 5 with a 4 bonus against armored.



The tests below are rudimentary, but that is okay. How much coverage you want will depend on the type of problem you are working on, who you work with, etc.

Here, this should be enough for our purposes.

In [5]:
def run_tests
  describe 'with no weapons' do
    assert('summary is correct') do
      ork = Ork.new('Richard').summary ==
        "This is a scruffy looking Ork, that goes by the name of Richard\nIt carries the following weapons:\nIt can deal a total damage of 0 with a 0 bonus against armored.\n"
    end
  end

  describe 'with a single weapon' do
    assert('summary is correct for a WOODEN weapon') do
      ork = Ork.new('Mary')
      ork.inventory.add(Weapon.new('Blade of Fury', 0))
      ork.summary() == 
        "This is a scruffy looking Ork, that goes by the name of Mary\nIt carries the following weapons:\n- Blade of Fury, damage: 2, bonus against armored: 0\nIt can deal a total damage of 2 with a 0 bonus against armored.\n"
    end
    
    assert('summary is correct for a BRONZE weapon') do
      ork = Ork.new('Joe')
      ork.inventory.add(Weapon.new('Knife of Summoning', 1))
      ork.summary() == 
        "This is a scruffy looking Ork, that goes by the name of Joe\nIt carries the following weapons:\n- Knife of Summoning, damage: 4, bonus against armored: 0\nIt can deal a total damage of 4 with a 0 bonus against armored.\n"
    end
     
    assert('summary is correct for an ADAMANTIUM weapon') do
      ork = Ork.new('Tobias')
      ork.inventory.add(Weapon.new('Holy Needle of Piercing', 2))
      ork.summary() == 
        "This is a scruffy looking Ork, that goes by the name of Tobias\nIt carries the following weapons:\n- Holy Needle of Piercing, damage: 3, bonus against armored: 4\nIt can deal a total damage of 3 with a 4 bonus against armored.\n"
    end
  end
  
  describe 'with multiple weapons' do
      assert('summary is correct for a WOODEN and an ADAMANTIUM weapon') do
      ork = Ork.new('Natalie')
      ork.inventory.add(Weapon.new('Blade of Fury', 0))
      ork.inventory.add(Weapon.new('Holy Needle of Piercing', 2))
      ork.summary() == 
        "This is a scruffy looking Ork, that goes by the name of Natalie\nIt carries the following weapons:\n- Blade of Fury, damage: 2, bonus against armored: 0\n- Holy Needle of Piercing, damage: 3, bonus against armored: 4\nIt can deal a total damage of 5 with a 4 bonus against armored.\n"
    end
  end
end

:run_tests

In [6]:
run_tests

### Extract Method

Extracting a method means the original method becomes more cohesive, in line with the Single Responsibility Pattern.

This can also help with DRYing up the code (DRY = Don't Repeat Yourself).

The inventory method is quite long and does a lot of things so let's see what we can do there.

In [7]:
class Ork
  def summary
    result = "This is a scruffy looking Ork, that goes by the name of #{name}\n"
    result += "It carries the following weapons:\n"
    
    total_damage, total_bonus_damage_against_armored = 0, 0
    inventory.weapons.each do |weapon|
      this_damage, this_bonus = damage_for(weapon)
      result += "- #{weapon.name}, damage: #{this_damage}, bonus against armored: #{this_bonus}\n"
      total_damage += this_damage
      total_bonus_damage_against_armored += this_bonus
    end
    
    result += "It can deal a total damage of #{total_damage} "
    result += "with a #{total_bonus_damage_against_armored} bonus against armored.\n"
  end
  
  def damage_for(weapon)
    this_bonus = 0
    case weapon.type
    when Weapon::WOODEN
      this_damage = 2
    when Weapon::BRONZE
      this_damage = 4
    when Weapon::ADAMANTIUM
      this_damage = 3
      this_bonus = 4
    end
    
    [this_damage, this_bonus]
  end   
end

:damage_for

In [8]:
run_tests

### Move Method

> In most cases a method should be on the object whose data it uses.

Again, this is the Single Responsibility Pattern at work.

The `damage_for` method relies on data that lives on the weapon, so let us move this method there. I also think it makes sense to split the method up (again, SRP).

In [9]:
class Weapon
  def damage
    case type
    when WOODEN
      return 2
    when BRONZE
      return 4
    when ADAMANTIUM
      return 3
    end
  end
  
  def bonus_damage_against_armored
    type == ADAMANTIUM ? 4 : 0
  end
end

class Ork
  def summary
    result = "This is a scruffy looking Ork, that goes by the name of #{name}\n"
    result += "It carries the following weapons:\n"
    
    total_damage, total_bonus_damage_against_armored = 0, 0
    inventory.weapons.each do |weapon|
      result += "- #{weapon.name}, damage: #{weapon.damage}, bonus against armored: #{weapon.bonus_damage_against_armored}\n"
      total_damage += weapon.damage
      total_bonus_damage_against_armored += weapon.bonus_damage_against_armored
    end
    
    result += "It can deal a total damage of #{total_damage} "
    result += "with a #{total_bonus_damage_against_armored} bonus against armored.\n"
  end 
end

:summary

In [10]:
run_tests

### Replace Temp with Query

Temporary variables can make methods longer, which make them harder to reason about. That makes a method also harder to test, though that consideration is secondary. There is also more mutable state which can lead to subtle bugs.

A long method (usually) goes against the SRP.

Let's extract total_bonus and total_damage.

In [11]:
class Inventory
  def damage
    damage = 0
    weapons.each do |weapon|
      damage += weapon.damage
    end
    damage
  end
  
  def bonus_damage_against_armored
    bonus_damage = 0
    weapons.each do |weapon|
      bonus_damage += weapon.bonus_damage_against_armored
    end
    bonus_damage
  end
end

class Ork
  def summary
    result = "This is a scruffy looking Ork, that goes by the name of #{name}\n"
    result += "It carries the following weapons:\n"
    
    inventory.weapons.each do |weapon|
      result += "- #{weapon.name}, damage: #{weapon.damage}, "
      result += "bonus against armored: #{weapon.bonus_damage_against_armored}\n"
    end
    
    result += "It can deal a total damage of #{inventory.damage} "
    result += "with a #{inventory.bonus_damage_against_armored} bonus against armored.\n"
  end
end

:summary

In [12]:
run_tests

### [Collection Closure Method / Collection Lambda](https://refactoring.com/catalog/replaceLoopWithCollectionClosureMethod.html)

I have seen this pattern so often and it is one of my favorites. Why are [Collection Pipelines](https://martinfowler.com/articles/collection-pipeline/) so great?

They make your code shorter, easier to reason about and are infinitely (within reason) composable.

In [13]:
class Inventory
  def damage
    weapons.inject(0) { |memo, weapon| memo += weapon.damage }
  end
  
  def bonus_damage_against_armored
    weapons.inject(0) { |memo, weapon| memo += weapon.bonus_damage_against_armored }
  end
end

:bonus_damage_against_armored

In [14]:
run_tests

### [Replace Type Code with Subclasses](https://refactoring.com/catalog/replaceTypeCodeWithSubclasses.html)

This also seems to deal with SRP and making our classes more cohesive. The advantages of doing this are that code is easier to change and extend.

Doing this is probably an overkill, but if there was more conditional logic relying on Weapon type, this would be more warranted.

In [15]:
class Weapon
  attr_reader :name
  
  def initialize(name)
    @name = name
  end
  
  def damage
    raise NotImplementedError
  end
  
  def bonus_damage_against_armored
    raise NotImplementedError
  end
end

class RegularWeapon < Weapon
  def bonus_damage_against_armored
    0
  end
end

class WoodenWeapon < RegularWeapon
  def damage
    2
  end
end

class BronzeWeapon < RegularWeapon
  def damage
    4
  end
end

class AdamantiumWeapon < Weapon
  def damage
    3
  end
  
  def bonus_damage_against_armored
    4
  end
end

:bonus_damage_against_armored

Unfortunately we need to change our tests to accomodate the change.

In [16]:
def run_tests
  describe 'with no weapons' do
    assert('summary is correct') do
      ork = Ork.new('Richard').summary ==
        "This is a scruffy looking Ork, that goes by the name of Richard\nIt carries the following weapons:\nIt can deal a total damage of 0 with a 0 bonus against armored.\n"
    end
  end

  describe 'with a single weapon' do
    assert('summary is correct for a WOODEN weapon') do
      ork = Ork.new('Mary')
      ork.inventory.add(WoodenWeapon.new('Blade of Fury'))
      ork.summary() == 
        "This is a scruffy looking Ork, that goes by the name of Mary\nIt carries the following weapons:\n- Blade of Fury, damage: 2, bonus against armored: 0\nIt can deal a total damage of 2 with a 0 bonus against armored.\n"
    end
    
    assert('summary is correct for a BRONZE weapon') do
      ork = Ork.new('Joe')
      ork.inventory.add(BronzeWeapon.new('Knife of Summoning'))
      ork.summary() == 
        "This is a scruffy looking Ork, that goes by the name of Joe\nIt carries the following weapons:\n- Knife of Summoning, damage: 4, bonus against armored: 0\nIt can deal a total damage of 4 with a 0 bonus against armored.\n"
    end
     
    assert('summary is correct for an ADAMANTIUM weapon') do
      ork = Ork.new('Tobias')
      ork.inventory.add(AdamantiumWeapon.new('Holy Needle of Piercing'))
      ork.summary() == 
        "This is a scruffy looking Ork, that goes by the name of Tobias\nIt carries the following weapons:\n- Holy Needle of Piercing, damage: 3, bonus against armored: 4\nIt can deal a total damage of 3 with a 4 bonus against armored.\n"
    end
  end
  
  describe 'with multiple weapons' do
      assert('summary is correct for a WOODEN and an ADAMANTIUM weapon') do
      ork = Ork.new('Natalie')
      ork.inventory.add(WoodenWeapon.new('Blade of Fury'))
      ork.inventory.add(AdamantiumWeapon.new('Holy Needle of Piercing'))
      ork.summary() == 
        "This is a scruffy looking Ork, that goes by the name of Natalie\nIt carries the following weapons:\n- Blade of Fury, damage: 2, bonus against armored: 0\n- Holy Needle of Piercing, damage: 3, bonus against armored: 4\nIt can deal a total damage of 5 with a 4 bonus against armored.\n"
    end
  end
end

:run_tests

In [17]:
run_tests

### Putting it all together

We did some monkey patching above. Let's copy all the code in one place to see what the final version of this would look like and whether it works (I restart the kernel before running the code below to be absolutely sure)

In [4]:
class Weapon
  attr_reader :name
  
  def initialize(name)
    @name = name
  end
  
  def damage
    raise NotImplementedError
  end
  
  def bonus_damage_against_armored
    raise NotImplementedError
  end
end

class RegularWeapon < Weapon
  def bonus_damage_against_armored
    0
  end
end

class WoodenWeapon < RegularWeapon
  def damage
    2
  end
end

class BronzeWeapon < RegularWeapon
  def damage
    4
  end
end

class AdamantiumWeapon < Weapon
  def damage
    3
  end
  
  def bonus_damage_against_armored
    4
  end
end

class Inventory
  attr_reader :weapons
  
  def initialize
    @weapons = []
  end
  
  def add(weapon)
    @weapons << weapon
  end
  
  def damage
    weapons.inject(0) { |memo, weapon| memo += weapon.damage }
  end
  
  def bonus_damage_against_armored
    weapons.inject(0) { |memo, weapon| memo += weapon.bonus_damage_against_armored }
  end
end

class Ork
  attr_reader :name, :inventory
  
  def initialize(name)
    @name = name
    @inventory = Inventory.new
  end
  
  def summary
    result = "This is a scruffy looking Ork, that goes by the name of #{name}\n"
    result += "It carries the following weapons:\n"
    
    inventory.weapons.each do |weapon|
      result += "- #{weapon.name}, damage: #{weapon.damage}, "
      result += "bonus against armored: #{weapon.bonus_damage_against_armored}\n"
    end
    
    result += "It can deal a total damage of #{inventory.damage} "
    result += "with a #{inventory.bonus_damage_against_armored} bonus against armored.\n"
  end
end

:summary

In [5]:
run_tests